In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import shutil
shutil.unpack_archive("./drive/My Drive/genres.tar.gz", "./drive/My Drive")

In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers
from keras import layers
import keras
from keras.models import Sequential
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


Now convert the audio data files into PNG format images or basically extracting the Spectrogram for every Audio.

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./drive/My Drive/genres/{g}'):
        songname = f'./drive/My Drive/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 576x576 with 0 Axes>

Creating a header for our CSV file.

In [ ]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

We will extract Mel-frequency cepstral coefficients (MFCC), Spectral Centroid, Zero Crossing Rate, Chroma Frequencies, and Spectral Roll-off.

In [ ]:
file = open('dataset.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./drive/My Drive/genres/{g}'):
        songname = f'./drive/My Drive/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        rmse = librosa.feature.rmse(y=y)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('dataset.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

loading CSV data, label encoding, feature scaling and data split into training and test set.

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
#Encoding the Labels
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)
#Scaling the Feature columns
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
#Dividing data into training and Testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Building an ANN model.

In [ ]:
model = Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
#model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_126 (Dense)            (None, 1024)              27648     
_________________________________________________________________
dense_127 (Dense)            (None, 512)               524800    
_________________________________________________________________
dense_128 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_129 (Dense)            (None, 128)               32896     
_________________________________________________________________
dense_130 (Dense)            (None, 64)                8256      
_________________________________________________________________
dense_131 (Dense)            (None, 10)                650       
Total params: 725,578
Trainable params: 725,578
Non-trainable params: 0
_______________________________________________

In [ ]:
classifier = model.fit(X_train,
                    y_train,
                    epochs=100,
                    batch_size=256)

Epoch 1/100
800/800 [==============================] - 0s 175us/step - loss: 7.0275e-05 - accuracy: 1.0000
Epoch 2/100
800/800 [==============================] - 0s 153us/step - loss: 6.9887e-05 - accuracy: 1.0000
Epoch 3/100
800/800 [==============================] - 0s 152us/step - loss: 6.9523e-05 - accuracy: 1.0000
Epoch 4/100
800/800 [==============================] - 0s 152us/step - loss: 6.9039e-05 - accuracy: 1.0000
Epoch 5/100
800/800 [==============================] - 0s 157us/step - loss: 6.8616e-05 - accuracy: 1.0000
Epoch 6/100
800/800 [==============================] - 0s 154us/step - loss: 6.8276e-05 - accuracy: 1.0000
Epoch 7/100
800/800 [==============================] - 0s 150us/step - loss: 6.7786e-05 - accuracy: 1.0000
Epoch 8/100
800/800 [==============================] - 0s 147us/step - loss: 6.7162e-05 - accuracy: 1.0000
Epoch 9/100
800/800 [==============================] - 0s 164us/step - loss: 6.6642e-05 - accuracy: 1.0000
Epoch 10/100
800/800 [===============

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)
print('test_loss: ',test_loss)

200/200 [==============================] - 0s 294us/step
test_acc:  0.6899999976158142
test_loss:  2.654512175321579


In [ ]:
!pip install h5py

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("./drive/My Drive/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("./drive/My Drive/model.h5")
print("Saved model to disk")


Saved model to disk


In [ ]:
#Validating our approach¶
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [ ]:
model = Sequential()
model.add(layers.Dense(2048, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 2048)              55296     
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dense_25 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_26 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_27 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_28 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_29 (Dense)             (None, 10)               

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 1s 855us/step - loss: 2.2941 - accuracy: 0.1250 - val_loss: 2.1525 - val_accuracy: 0.2950
Epoch 2/30
600/600 [==============================] - 0s 472us/step - loss: 2.0803 - accuracy: 0.3700 - val_loss: 1.9381 - val_accuracy: 0.3050
Epoch 3/30
600/600 [==============================] - 0s 489us/step - loss: 1.7630 - accuracy: 0.4083 - val_loss: 1.7563 - val_accuracy: 0.3550
Epoch 4/30
600/600 [==============================] - 0s 529us/step - loss: 1.5364 - accuracy: 0.4350 - val_loss: 1.6104 - val_accuracy: 0.4050
Epoch 5/30
600/600 [==============================] - 0s 481us/step - loss: 1.3574 - accuracy: 0.5050 - val_loss: 1.5763 - val_accuracy: 0.4700
Epoch 6/30
600/600 [==============================] - 0s 491us/step - loss: 1.3676 - accuracy: 0.5183 - val_loss: 1.6820 - val_accuracy: 0.3900
Epoch 7/30
600/600 [==============================] - 0s 489us/step - loss: 1.2256 - accur

In [ ]:
results = model.evaluate(X_test, y_test)


200/200 [==============================] - 0s 303us/step


In [ ]:
#Predictions on Test Data¶
predictions = model.predict(X_test)